In [ ]:
from __future__ import print_function

import sys
import os
from IPython.display import display, clear_output

sys.path.insert(0, '/spark/python')
sys.path.insert(0, '/spark/python/lib/py4j-0.10.7-src.zip')
os.environ['SPARK_HOME'] = '/spark'

import pyspark
conf = pyspark.SparkConf()
conf.setMaster("spark://spark-master:7077")
conf.set("spark.driver.memory","1g")
conf.set("spark.executor.memory","1g")
conf.set("spark.num.executors","2")
conf.set("spark.executor.cores","1")
sc = pyspark.SparkContext(conf=conf)
print(sc)

from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext,SparkSession

ssc = pyspark.streaming.StreamingContext(sc, 5)
ssc.checkpoint("checkpoint_TwitterApp")
print(ssc)

In [ ]:
HOST = '17.18.0.2'  # IP address of spark-master
PORT = 9009  

def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

def getSparkSessionInstance(sparkConf):
    if ('sparkSessionSingletonInstance' not in globals()):
        globals()['sparkSessionSingletonInstance'] = SparkSession\
            .builder\
            .config(conf=sparkConf)\
            .getOrCreate()
    return globals()['sparkSessionSingletonInstance']

def send_df_to_dashboard(df):
    # extract the hashtags from dataframe and convert them into array
    top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
    # extract the counts from dataframe and convert them into array
    tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
    # initialize and send the data through REST API
    url = 'http://' + HOST + ':5001/updateData'
    request_data = {'label': str(top_tags), 'data': str(tags_count)}
    response = requests.post(url, data=request_data)

In [ ]:
def process_rdd(time, rdd):
    #clear_output()
    print("----------- %s -----------" % str(time))
    try:
        sql_context = getSparkSessionInstance(rdd.context.getConf())
        print(sql_context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: pyspark.sql.Row(hashtag=w[0], hashtag_count=w[1]))
        print(row_rdd.take(5))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd, ("hashtag","hashtag_count"))
        #hashtag_df.sql_ctx.sparkSession._jsparkSession = rdd.context._jsparkSession
        #hashtag_df._sc = rdd.context._sc
        print(hashtags_df.show())
        print("done data frame")
        # Register the dataframe as table
        hashtags_df.createOrReplaceTempView("hashtags")
        print("Finish creating table ...")
        # get the top 10 hashtags from the table using SQL and print them
        sql_stmt = "select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10"
        hashtag_counts_df = sql_context.sql(sql_stmt)
        #hashtag_counts_df.show()

        # call this method to prepare top 10 hashtags DF and send them
        send_df_to_dashboard(hashtag_counts_df)
    except:
        d = sys.exc_info()
        e = sys.exc_info()[1]
        #print("Error: %s" % e)
        print(e)

In [ ]:
lines = ssc.socketTextStream(HOST, PORT)
words = lines.flatMap(lambda line: line.split(" "))

hashtags = words.filter(lambda w: '#' in w).map(lambda w: (w, 1))
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
tags_totals.foreachRDD(process_rdd)

ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()
sc.stop()